In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import os
from datetime import date, time,datetime

In [2]:
app = Flask(__name__)
received_data = []

In [19]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

import pytz
import json

INDIA = pytz.timezone('Asia/Kolkata')


def get_calendar_service(token_path):
    with open(token_path, 'r') as f:
        token_data = json.load(f)
    creds = Credentials.from_authorized_user_info(token_data)
    return build('calendar', 'v3', credentials=creds)

def get_events(service, start_time, end_time):
    events_result = service.events().list(
        calendarId='primary',
        timeMin=start_time.isoformat(),
        timeMax=end_time.isoformat(),
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    return events_result.get('items', [])

def parse_dt(dt_str):
    return datetime.datetime.fromisoformat(dt_str.replace('Z', '+00:00'))

def extract_off_hours(events, target_date):
    """
    Try to extract off-hours if there's a full-day or marked 'transparent' event.
    Example: an event with summary 'Off hours' from 18:00 to next day 09:00
    """
    for e in events:
        summary = e.get('summary', '').lower()
        if "off" in summary or "non-working" in summary:
            start = parse_dt(e['start'].get('dateTime', e['start'].get('date')))
            end = parse_dt(e['end'].get('dateTime', e['end'].get('date')))
            if start.date() == target_date:
                return start.time(), end.time()
    return None, None

def merge_busy_slots(events):
    busy = []
    for e in events:
        start = parse_dt(e['start'].get('dateTime', e['start'].get('date')))
        end = parse_dt(e['end'].get('dateTime', e['end'].get('date')))
        busy.append((start, end))
    return busy

def compute_available_slots(work_start, work_end, busy_slots):
    busy_slots.sort()
    available = []
    current = work_start
    for start, end in busy_slots:
        if current < start:
            available.append((current, start))
        current = max(current, end)
    if current < work_end:
        available.append((current, work_end))
    return available

def get_user_availability(email, token_path, date):
    if date.weekday() >= 5:  # Skip weekends
        return []

    service = get_calendar_service(token_path)
    day_start = INDIA.localize(datetime.datetime.combine(date, datetime.time(0, 0)))
    day_end = INDIA.localize(datetime.datetime.combine(date, datetime.time(23, 59)))

    events = get_events(service, day_start, day_end)

    # Detect off-hours → working hours
    off_start, off_end = extract_off_hours(events, date)

    if off_start and off_end:
        # Wrap-around case (e.g. 20:00 to 11:00)
        if off_end < off_start:
            work_start = datetime.datetime.combine(date, off_end)
            work_end = datetime.datetime.combine(date, off_start)
        else:
            # Non-wraparound — normal (18:00 to 09:00) = work 9 AM to 6 PM
            work_start = datetime.datetime.combine(date, off_end)
            work_end = datetime.datetime.combine(date, off_start)
    else:
        # Default to 9 AM – 6 PM if not detected
        work_start = datetime.datetime.combine(date, datetime.time(9, 0))
        work_end = datetime.datetime.combine(date, datetime.time(18, 0))

    work_start = INDIA.localize(work_start)
    work_end = INDIA.localize(work_end)

    busy_slots = merge_busy_slots(events)
    return compute_available_slots(work_start, work_end, busy_slots)

def getAttendeesAvailableSlots(token_dictionary, target_date: datetime.date):
    all_avail = {}

    for email, token_path in token_dictionary.items():
        try:
            # Get availability slots as datetime tuples
            slots = get_user_availability(email, token_path, target_date)

            # Convert each slot to ["start", "end"] in ISO format
            iso_slots = [
                [start.isoformat(), end.isoformat()]
                for start, end in slots
            ]

            all_avail[email] = iso_slots

        except Exception as e:
            print(all_avail)
            print(f"❌ Error for {email}: {e}")
            all_avail[email] = []

    return all_avail


In [20]:
def find_token_files_with_emails(folder_path: str) -> dict:
    token_map = {}
    for file in os.listdir(folder_path):
        if file.endswith(".token"):
            token_path = os.path.join(folder_path, file)
            email_user = file.replace(".token", "")
            email = f"{email_user}@gmail.com"
            token_map[email] = token_path
    return token_map

In [22]:
token_files = find_token_files_with_emails("/home/user/Keys/")
print(token_files)

{'userthree.amd@gmail.com': '/home/user/Keys/userthree.amd.token', 'userone.amd@gmail.com': '/home/user/Keys/userone.amd.token', 'usertwo.amd@gmail.com': '/home/user/Keys/usertwo.amd.token'}


In [23]:
token_dictionary = find_token_files_with_emails("/home/user/Keys/")
# print(token_dictionary)
# get the attendees available slots
target_date = date(2025, 7, 15)
slots = getAttendeesAvailableSlots(token_dictionary, target_date)
print(json.dumps(slots, indent=2))

{}
❌ Error for userthree.amd@gmail.com: type object 'datetime.datetime' has no attribute 'datetime'
{'userthree.amd@gmail.com': []}
❌ Error for userone.amd@gmail.com: type object 'datetime.datetime' has no attribute 'datetime'
{'userthree.amd@gmail.com': [], 'userone.amd@gmail.com': []}
❌ Error for usertwo.amd@gmail.com: type object 'datetime.datetime' has no attribute 'datetime'
{
  "userthree.amd@gmail.com": [],
  "userone.amd@gmail.com": [],
  "usertwo.amd@gmail.com": []
}


In [7]:


BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = OpenAIModel("Qwen3-30B-A3B", provider=provider)

import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output


from datetime import datetime
from pydantic_ai import Tool          
@Tool
def get_current_date() -> str:
    """Return the current date/time as an ISO-formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
from pydantic_ai import Agent

agent = Agent(
    model=agent_model,
    tools=[get_current_date],
    system_prompt = """
    You are an expert assistant for parsing meeting requests. Your task is to extract key details into a structured JSON format.
The current date for context is {request_datetime}.
- If the user specifies a range (e.g., "next week", "between Monday and Wednesday"), provide a start and end date.
- If the user specifies a single day (e.g., "tomorrow", "on Tuesday"), the start and end date will be the same.
- A "week" starts on Monday and ends on Friday.
- "time_preference" can be 'morning', 'afternoon', 'evening', or a specific time like '2 PM'.

You MUST return ONLY a valid JSON object string. Do not add any other text.

Example 1:
User request: "Let's meet sometime next week for an hour to discuss the project."
Current Date: 2025-07-18 (Friday)
Output:
{{
  "meeting_summary": "discuss the project",
  "duration_minutes": 60,
  "search_start_date": "2025-07-21",
  "search_end_date": "2025-07-25",
  "time_preference": null
}}

Example 2:
User request: "Can we find 30 minutes on Monday afternoon?"
Current Date: 2025-07-18 (Friday)
Output:
{{
  "meeting_summary": "find 30 minutes",
  "duration_minutes": 30,
  "search_start_date": "2025-07-21",
  "search_end_date": "2025-07-21",
  "time_preference": "afternoon"
}}
    """
)


In [8]:
!curl http://localhost:8000/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

{"object":"list","data":[{"id":"Qwen3-30B-A3B","object":"model","created":1752397015,"owned_by":"vllm","root":"Qwen/Qwen3-30B-A3B","parent":null,"max_model_len":40960,"permission":[{"id":"modelperm-3516b2abc8f54f50b8ba73f0aabb09fd","object":"model_permission","created":1752397015,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [9]:
# await run_async("""{
#     "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
#     "Datetime": "09-07-2025T12:34:55",
#     "Location": "IIT Mumbai",
#     "From": "userone.amd@gmail.com",
#     "Attendees": [
#         {
#             "email": "usertwo.amd@gmail.com"
#         },
#         {
#             "email": "userthree.amd@gmail.com"
#         }
#     ],
#     "Subject": "Agentic AI Project Status Update",
#     "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
# }""")

In [10]:
def getTimeAndDateUsingEmail(email_content, email_time, attendees_available_slots):
    input_prompt = f"""EmailSentTime: {email_time}
    EmailContent: {email_content}
    AvailableSlots: {attendees_available_slots}
    """
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    return loop.run_until_complete(run_async(input_prompt))
    

In [11]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()

    # Extract EmailContent and EmailSentTime
    email_content = data.get("EmailContent", "")
    email_time = data.get("Datetime", "")
    
    # Extract list of attendee emails
    attendees = [attendee.get("email") for attendee in data.get("Attendees", [])]

    token_dictionary = find_token_files_with_emails("/home/user/Keys/")
    attendees_available_slots = getAttendeesAvailableSlots(token_dictionary, target_date)

    # Step 1: Use LLM to extract StartTime & EndTime
    time_date_response = getTimeAndDateUsingEmail(email_content, email_time, attendees_available_slots)

    # Step 2: Dynamically determine target date
    # target_date = datetime.fromisoformat(time_date_response["StartTime"]).date()
    
    return jsonify([attendees,time_date_response])

    # # Step 4: Compare extracted meeting time with attendee availability
    # availability_dictionary = check_time_matching(attendees_available_slots, time_date_response)

    # Continue with logic: either confirm or suggest alternate slot


In [12]:
def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [13]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.176.251:5000
Press CTRL+C to quit


{}
❌ Error for userthree.amd@gmail.com: type object 'datetime.datetime' has no attribute 'datetime'
{'userthree.amd@gmail.com': []}
❌ Error for userone.amd@gmail.com: type object 'datetime.datetime' has no attribute 'datetime'
{'userthree.amd@gmail.com': [], 'userone.amd@gmail.com': []}
❌ Error for usertwo.amd@gmail.com: type object 'datetime.datetime' has no attribute 'datetime'


42.107.78.33 - - [13/Jul/2025 08:57:12] "POST /receive HTTP/1.1" 200 -
95.215.0.144 - - [13/Jul/2025 08:58:12] code 400, message Bad request version ("¯nãY»bhlÿ(=':©\x82ÙoÈ¢×\x93\x98´ï\x80å¹\x90\x00(À")
95.215.0.144 - - [13/Jul/2025 08:58:12] "\x16\x03\x02\x01o\x01\x00\x01k\x03\x02RHÅ\x1a#÷:Nßâ´\x82/ÿ\x09T\x9f§Äy°hÆ\x13\x8c¤\x1c="á\x1a\x98 \x84´,\x85¯nãY»bhlÿ(=':©\x82ÙoÈ¢×\x93\x98´ï\x80å¹\x90\x00(À" 400 -
176.32.195.85 - - [13/Jul/2025 08:58:27] code 400, message Bad request version ('À\x13À')
176.32.195.85 - - [13/Jul/2025 08:58:27] "\x16\x03\x01\x05¨\x01\x00\x05¤\x03\x03¬þÓñm¸ìì"´\x16òð\x82\x9ff«ÞÇ²\x8f¿\x84gÚ`¥âÏOÐÉ ÃT-÷\x10\x99\x14\x91kÙ¥\x16BíÔ\x06,i\x86ÒO0liÈ"x\x7f6/E\x0d\x00\x1aÀ+À/À,À0Ì©Ì¨À\x09À\x13À" 400 -
176.32.195.85 - - [13/Jul/2025 08:58:27] "GET /v2/_catalog HTTP/1.1" 404 -


In [14]:
await run_async("EmailSentTime: 09-07-2025T12:34:55,EmailContent:Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.")

'\n\n{\n  "meeting_summary": "discuss the status of Agentic AI Project",\n  "duration_minutes": 30,\n  "search_start_date": "2025-09-10",\n  "search_end_date": "2025-09-10",\n  "time_preference": null\n}'